In [14]:
from data.dicts import DICTS
from nltk.stem import WordNetLemmatizer
from transformers import pipeline
import numpy as np

In [41]:
# BERT maskierten satz geben
def bert_predict(sentence):
    model = pipeline('fill-mask', model='bert-base-uncased')
    return model(sentence)

# verb stemmen
def get_verb_lemma(verb: str) -> str:
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(verb, pos='v')

# mit dicts vergleichen
def predict_class(bert_predictions, class_dicts):
    scores = []
    score = 0
    for d in class_dicts:
        for prediction in bert_predictions:
            try:
                verb = get_verb_lemma(prediction['token_str'])
                score += d[0][verb] * prediction['score']
            except:
                continue

        scores.append((score))
        score = 0
    
    # softmax and sort scores and add labels
    softmaxed_scores = softmax(np.array(scores))
    labeled_scores = []
    for score, label in zip(softmaxed_scores, class_dicts):
        labeled_scores.append([label[1], score])
    labeled_scores = sorted(labeled_scores, key=lambda item: item[1], reverse=True)
    return labeled_scores


def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [42]:
sentence = 'The second simulation was [MASK] from the uncomplexed x-ray structure after insertion of the ligand into the binding site.'
pred = bert_predict(sentence)
scores = predict_class(pred, DICTS)

print(scores)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[['PP', 0.7866813436862404], ['CE', 0.056809240624320814], ['IA', 0.03209467244996376], ['MC', 0.024882948647895028], ['EO', 0.024882948647895028], ['CT', 0.024882948647895028], ['ED', 0.024882948647895028], ['CW', 0.024882948647895028]]
